# Detecção Avançada de Anomalias em Experimentos de Noisy Neighbors

Este notebook demonstra o uso das funcionalidades de detecção de anomalias para análise de experimentos de noisy neighbors no Kubernetes.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ajustar o path para importar os módulos do pipeline
sys.path.append(os.path.abspath('../..'))

# Importar módulos necessários
from pipeline.data_processing.consolidation import load_experiment_data, select_tenants
from pipeline.data_processing.time_normalization import normalize_time
from pipeline.data_processing.aggregation import aggregate_by_time
from pipeline.analysis.anomaly_detection import detect_anomalies_isolation_forest, \
                                               detect_anomalies_local_outlier_factor, \
                                               detect_change_points, \
                                               detect_pattern_changes, \
                                               detect_anomalies_ensemble
from pipeline.visualization.plots import plot_metric_with_anomalies, \
                                        plot_multivariate_anomalies, \
                                        plot_change_points, \
                                        plot_metric_by_phase, plot_phase_comparison # Added for EDA
from pipeline.config import METRIC_DISPLAY_NAMES, TENANT_COLORS, VISUALIZATION_CONFIG # Added for EDA

# Configurar visualização
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

## 1. Carregar e Preparar os Dados

Primeiro vamos carregar os dados de um experimento de exemplo e preparar os dados para análise de anomalias.

In [ ]:
# Caminho para os dados do experimento
experiment_path = '../../demo-data/demo-experiment-3-rounds/'

# Carregar dados do experimento
metrics_data, experiment_info = load_experiment_data(experiment_path)

print(f"Experimento carregado: {experiment_info['name']}")
print(f"Métricas disponíveis: {list(metrics_data.keys())}")
print(f"Tenants disponíveis: {experiment_info['tenants']}")
print(f"Fases do experimento: {experiment_info['phases']}")

In [ ]:
# Selecionar métricas para análise
selected_metrics = ['cpu_usage', 'memory_usage']
selected_data = {k: metrics_data[k] for k in selected_metrics if k in metrics_data}

# Selecionar tenants específicos (opcional)
tenants_of_interest = ['tenant-a', 'tenant-b']
selected_data = {k: select_tenants(df, tenants_of_interest) for k, df in selected_data.items()}

# Normalizar o tempo para facilitar a análise
selected_data = {k: normalize_time(df, experiment_info) for k, df in selected_data.items()}

# Adicionar coluna de tempo decorrido em minutos para facilitar visualizações
for k, df in selected_data.items():
    df['elapsed_minutes'] = (df['datetime'] - df['datetime'].min()).dt.total_seconds() / 60

## 1.A. Exploratory Data Analysis (EDA)

This section performs a basic exploratory data analysis on the loaded and prepared data.
We will focus on a specific metric to understand its behavior across different tenants and experiment phases.

In [ ]:
# Define a metric for the EDA section
# You can change this to any of the selected_metrics
if selected_metrics:
    metric_name_for_eda = selected_metrics[0]
else:
    metric_name_for_eda = 'cpu_usage' # Fallback if selected_metrics is empty

print(f"--- EDA for metric: {metric_name_for_eda} ---")

if metric_name_for_eda in selected_data:
    df_eda = selected_data[metric_name_for_eda]
    print("\nData Shape (for EDA metric):")
    print(df_eda.shape)
    print("\nData Types (for EDA metric):")
    print(df_eda.dtypes)
    print("\nDescriptive Statistics (for 'value' column of EDA metric):")
    if 'value' in df_eda.columns and not df_eda.empty:
        print(df_eda['value'].describe())
    elif df_eda.empty:
        print("DataFrame for EDA metric is empty, cannot show descriptive statistics.")
    else:
        print("'value' column not found in EDA metric DataFrame.")
    print("\nFirst 5 rows (for EDA metric):")
    print(df_eda.head())
else:
    print(f"Metric {metric_name_for_eda} not found in selected_data. Skipping EDA for this metric.")

### Visualize Metric Over Time by Phase (EDA)

Using `plot_metric_by_phase` to visualize how the selected metric changes over time for each tenant, with phase distinctions.

In [ ]:
if metric_name_for_eda in selected_data and not selected_data[metric_name_for_eda].empty:
    df_eda = selected_data[metric_name_for_eda]
    fig_metric_by_phase_eda = plot_metric_by_phase(
        df=df_eda,
        metric_name=metric_name_for_eda,
        time_column='elapsed_minutes', # or 'datetime'
        show_phase_markers=True
    )
    plt.suptitle(f"EDA: {METRIC_DISPLAY_NAMES.get(metric_name_for_eda, metric_name_for_eda)} Over Time by Phase", fontsize=16)
    plt.show()
else:
    print(f"DataFrame for {metric_name_for_eda} is empty or not found. Skipping EDA plot.")

**Insights from EDA plot:**
- Observe the baseline behavior of the metric for each tenant.
- Identify how the metric changes during the 'Attack' phase (if applicable based on data).
- Assess the recovery during the 'Recovery' phase (if applicable).

### Phase Comparison Plot (EDA)

To compare phases, we first need to aggregate the data by tenant and phase to get summary statistics (e.g., mean, std) for the metric in each phase.

In [ ]:
if metric_name_for_eda in selected_data and not selected_data[metric_name_for_eda].empty and 'value' in selected_data[metric_name_for_eda].columns:
    df_eda = selected_data[metric_name_for_eda]
    # Aggregate data: calculate mean and std of 'value' for each tenant and phase
    aggregated_data_eda = df_eda.groupby(['tenant', 'phase'])['value'].agg(['mean', 'std']).reset_index()

    print(f"\nAggregated Data for Phase Comparison (EDA metric: {metric_name_for_eda}):")
    print(aggregated_data_eda.head())

    fig_phase_comparison_eda = plot_phase_comparison(
        df=aggregated_data_eda,
        metric_name=metric_name_for_eda,
        value_column='mean',
        error_column='std' # Optional: if you want error bars
    )
    plt.suptitle(f"EDA: Phase Comparison for {METRIC_DISPLAY_NAMES.get(metric_name_for_eda, metric_name_for_eda)}", fontsize=16)
    plt.show()
else:
    print(f"DataFrame for {metric_name_for_eda} is empty or 'value' column missing. Skipping EDA phase comparison plot.")

**Insights from Phase Comparison Plot (EDA):**
- Directly compare the average (or other aggregate) metric value across phases for each tenant.
- Quantify the impact of different phases relative to each other.

This concludes the initial exploratory data analysis. The subsequent sections will delve into more advanced anomaly detection techniques.

## 2. Detecção de Anomalias usando Isolation Forest

O Isolation Forest é um algoritmo eficiente para detectar anomalias baseado na premissa de que anomalias são mais fáceis de isolar do que observações normais.

In [ ]:
# Selecionar métrica para análise
metric_name = 'cpu_usage'
df = selected_data[metric_name].copy()

# Aplicar Isolation Forest para detectar anomalias
df_with_anomalies = detect_anomalies_isolation_forest(
    df, 
    metric_column='value', 
    contamination=0.05,  # Proporção esperada de anomalias
    group_by=['tenant', 'phase']  # Agrupar por tenant e fase
)

# Verificar quantas anomalias foram detectadas
anomaly_count = df_with_anomalies['is_anomaly_if'].sum()
total_points = len(df_with_anomalies)
print(f"Detectadas {anomaly_count} anomalias em {total_points} pontos ({anomaly_count/total_points*100:.2f}%)")

In [ ]:
# Visualizar os resultados para um tenant específico
tenant = 'tenant-a'
tenant_data = df_with_anomalies[df_with_anomalies['tenant'] == tenant]

plt.figure(figsize=(14, 7))
plt.scatter(tenant_data['elapsed_minutes'], tenant_data['value'], 
            c=tenant_data['is_anomaly_if'].map({True: 'red', False: 'blue'}),
            alpha=0.6, s=30)
plt.title(f'Detecção de Anomalias com Isolation Forest - {metric_name.upper()} para {tenant}')
plt.xlabel('Tempo Decorrido (minutos)')
plt.ylabel('Valor')
plt.colorbar(plt.cm.ScalarMappable(cmap='coolwarm'), 
             label='Score de Anomalia')
plt.grid(True)
plt.tight_layout()
plt.show()

## 3. Detecção de Anomalias usando Local Outlier Factor (LOF)

O LOF identifica anomalias baseando-se na densidade local dos pontos de dados.

In [ ]:
# Aplicar LOF para detectar anomalias
df_with_lof = detect_anomalies_local_outlier_factor(
    df, 
    metric_column='value', 
    n_neighbors=20,  # Número de vizinhos a considerar
    contamination=0.05,  # Proporção esperada de anomalias
    group_by=['tenant', 'phase']  # Agrupar por tenant e fase
)

# Verificar quantas anomalias foram detectadas
anomaly_count = df_with_lof['is_anomaly_lof'].sum()
total_points = len(df_with_lof)
print(f"Detectadas {anomaly_count} anomalias em {total_points} pontos ({anomaly_count/total_points*100:.2f}%)")

In [ ]:
# Visualizar os resultados para um tenant específico
tenant = 'tenant-a'
tenant_data = df_with_lof[df_with_lof['tenant'] == tenant]

plt.figure(figsize=(14, 7))
plt.scatter(tenant_data['elapsed_minutes'], tenant_data['value'], 
            c=tenant_data['is_anomaly_lof'].map({True: 'red', False: 'blue'}),
            alpha=0.6, s=30)
plt.title(f'Detecção de Anomalias com LOF - {metric_name.upper()} para {tenant}')
plt.xlabel('Tempo Decorrido (minutos)')
plt.ylabel('Valor')
plt.grid(True)
plt.tight_layout()
plt.show()

## 4. Detecção de Pontos de Mudança

Os pontos de mudança são momentos nos quais as propriedades estatísticas de uma série temporal mudam significativamente.

In [ ]:
# Detectar pontos de mudança
df_with_changes, change_info = detect_change_points(
    df, 
    metric_column='value',
    time_column='elapsed_minutes',
    method='pelt',  # Algoritmo PELT (Pruned Exact Linear Time)
    model='l2',  # Modelo de custo L2 (erro quadrático)
    min_size=10,  # Tamanho mínimo de segmento
    penalty=3,  # Penalidade para controlar o número de pontos
    group_by=['tenant']  # Agrupar por tenant
)

# Exibir informações sobre os pontos detectados
for group, info in change_info.items():
    if isinstance(group, tuple):
        group_name = group[0]  # Para tuples (quando group_by é usado)
    else:
        group_name = group
    print(f"Grupo: {group_name}")
    print(f"  Número de pontos de mudança: {info['n_change_points']}")
    if info['change_point_times']:
        print(f"  Tempos dos pontos de mudança: {[round(t, 2) for t in info['change_point_times']]}")
    print()

In [ ]:
# Visualizar os pontos de mudança
tenant = 'tenant-a'
tenant_data = df_with_changes[df_with_changes['tenant'] == tenant]

plt.figure(figsize=(14, 7))
plt.plot(tenant_data['elapsed_minutes'], tenant_data['value'], 'b-', alpha=0.7)

# Destacar pontos de mudança
change_points = tenant_data[tenant_data['is_change_point']]
plt.scatter(change_points['elapsed_minutes'], change_points['value'], 
            color='red', s=80, marker='o', label='Pontos de Mudança')

plt.title(f'Detecção de Pontos de Mudança - {metric_name.upper()} para {tenant}')
plt.xlabel('Tempo Decorrido (minutos)')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 5. Detecção de Mudanças de Padrão

Essa abordagem identifica mudanças nos padrões de comportamento usando clustering de séries temporais.

In [ ]:
# Preparar múltiplas métricas para análise de padrões
metrics_to_analyze = ['cpu_usage', 'memory_usage']
df_multi = pd.DataFrame()

for metric in metrics_to_analyze:
    if metric in selected_data:
        temp_df = selected_data[metric].copy()
        temp_df = temp_df.rename(columns={'value': metric})
        
        # Mesclar com o DataFrame principal
        if df_multi.empty:
            df_multi = temp_df[['tenant', 'phase', 'round', 'datetime', 'elapsed_minutes', metric]]
        else:
            df_multi = pd.merge(
                df_multi, 
                temp_df[['tenant', 'datetime', metric]], 
                on=['tenant', 'datetime'], 
                how='outer'
            )

# Verificar o DataFrame resultante
print(f"Formato do DataFrame: {df_multi.shape}")
df_multi.head()

In [ ]:
# Detectar mudanças de padrão
pattern_changes = detect_pattern_changes(
    df_multi,
    metrics=metrics_to_analyze,
    time_column='elapsed_minutes',
    window_size=15,  # Tamanho da janela para capturar padrões
    n_clusters=3,    # Número de clusters de padrões
    group_by=['tenant']  # Agrupar por tenant
)

# Exibir mudanças de padrão detectadas
if not pattern_changes.empty:
    print(f"Detectadas {len(pattern_changes)} mudanças de padrão")
    display(pattern_changes)
else:
    print("Nenhuma mudança de padrão detectada ou dados insuficientes para análise")

## 6. Abordagem de Ensemble para Detecção de Anomalias

Combinar múltiplos algoritmos pode melhorar a precisão da detecção de anomalias.

In [ ]:
# Aplicar ensemble de detecção de anomalias
df_ensemble, change_info_ensemble = detect_anomalies_ensemble(
    df, 
    metric_column='value',
    time_column='elapsed_minutes',
    contamination=0.05,
    group_by=['tenant', 'phase']
)

# Verificar resultados
anomaly_count = df_ensemble['is_anomaly'].sum()
total_points = len(df_ensemble)
print(f"Detectadas {anomaly_count} anomalias em {total_points} pontos ({anomaly_count/total_points*100:.2f}%)")

In [ ]:
# Visualizar os resultados do ensemble para um tenant específico
tenant = 'tenant-a'
tenant_data = df_ensemble[df_ensemble['tenant'] == tenant]

plt.figure(figsize=(14, 7))

# Plotar os valores
plt.plot(tenant_data['elapsed_minutes'], tenant_data['value'], 'b-', alpha=0.5, label='Valores')

# Destacar anomalias do método ensemble
anomalies = tenant_data[tenant_data['is_anomaly']]
plt.scatter(anomalies['elapsed_minutes'], anomalies['value'], 
            color='red', s=50, marker='o', label='Anomalias (Ensemble)')

# Destacar pontos de mudança
change_points = tenant_data[tenant_data['is_change_point']]
plt.scatter(change_points['elapsed_minutes'], change_points['value'], 
            color='purple', s=80, marker='X', label='Pontos de Mudança')

plt.title(f'Detecção com Ensemble - {metric_name.upper()} para {tenant}')
plt.xlabel('Tempo Decorrido (minutos)')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 7. Comparação entre Diferentes Tipos de Anomalias

Analisar diferenças entre os tipos de anomalias detectadas por cada algoritmo.

In [ ]:
# Contabilizar os diferentes tipos de anomalias
anomaly_types = pd.DataFrame({
    'Isolation Forest': df_ensemble['is_anomaly_if'].sum(),
    'LOF': df_ensemble['is_anomaly_lof'].sum(),
    'Ensemble': df_ensemble['is_anomaly'].sum(),
    'Pontos de Mudança': df_ensemble['is_change_point'].sum()
}, index=['Total'])

# Calcular estatísticas por tenant
tenant_stats = []
for tenant in df_ensemble['tenant'].unique():
    tenant_data = df_ensemble[df_ensemble['tenant'] == tenant]
    tenant_stats.append({
        'Tenant': tenant,
        'Isolation Forest': tenant_data['is_anomaly_if'].sum(),
        'LOF': tenant_data['is_anomaly_lof'].sum(),
        'Ensemble': tenant_data['is_anomaly'].sum(),
        'Pontos de Mudança': tenant_data['is_change_point'].sum(),
        'Total de Pontos': len(tenant_data)
    })

tenant_anomaly_stats = pd.DataFrame(tenant_stats).set_index('Tenant')

# Exibir estatísticas
print("Estatísticas Globais:")
display(anomaly_types)

print("\nEstatísticas por Tenant:")
display(tenant_anomaly_stats)

In [ ]:
# Visualizar a distribuição dos scores de anomalia
plt.figure(figsize=(14, 7))

# Criar um DataFrame para a visualização
scores_df = pd.DataFrame({
    'Isolation Forest': df_ensemble['anomaly_score_if'],
    'LOF': df_ensemble['anomaly_score_lof'],
    'Ensemble': df_ensemble['anomaly_score_combined']
})

# Plotar histograma
scores_df.hist(bins=50, alpha=0.7, figsize=(14, 5))
plt.suptitle('Distribuição dos Scores de Anomalia', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Plotar boxplots para comparar distribuições
plt.figure(figsize=(10, 6))
sns.boxplot(data=scores_df)
plt.title('Comparação dos Scores de Anomalia entre Algoritmos')
plt.ylabel('Score de Anomalia')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Relação entre Anomalias e Fases do Experimento

Analisar como a distribuição de anomalias varia entre as diferentes fases do experimento.

In [ ]:
# Análise por fase
phase_stats = []
for phase in df_ensemble['phase'].unique():
    phase_data = df_ensemble[df_ensemble['phase'] == phase]
    phase_stats.append({
        'Fase': phase,
        'Total de Pontos': len(phase_data),
        'Anomalias (IF)': phase_data['is_anomaly_if'].sum(),
        'Anomalias (LOF)': phase_data['is_anomaly_lof'].sum(),
        'Anomalias (Ensemble)': phase_data['is_anomaly'].sum(),
        'Pontos de Mudança': phase_data['is_change_point'].sum(),
        '% Anomalias': phase_data['is_anomaly'].sum() / len(phase_data) * 100
    })

phase_anomaly_stats = pd.DataFrame(phase_stats).set_index('Fase')
display(phase_anomaly_stats)

In [ ]:
# Visualizar a distribuição de anomalias por fase
plt.figure(figsize=(12, 8))

# Preparar dados para visualização
phases = phase_anomaly_stats.index
if_anomalies = phase_anomaly_stats['Anomalias (IF)']
lof_anomalies = phase_anomaly_stats['Anomalias (LOF)']
ensemble_anomalies = phase_anomaly_stats['Anomalias (Ensemble)']

# Plotar barras agrupadas
x = np.arange(len(phases))
width = 0.25

plt.bar(x - width, if_anomalies, width, label='Isolation Forest')
plt.bar(x, lof_anomalies, width, label='LOF')
plt.bar(x + width, ensemble_anomalies, width, label='Ensemble')

plt.xlabel('Fase')
plt.ylabel('Número de Anomalias')
plt.title('Distribuição de Anomalias por Fase')
plt.xticks(x, phases)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Porcentagem de anomalias por fase
plt.figure(figsize=(10, 6))
plt.bar(phases, phase_anomaly_stats['% Anomalias'], color='darkred')
plt.xlabel('Fase')
plt.ylabel('% de Pontos Anômalos')
plt.title('Porcentagem de Anomalias por Fase')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Análise de Correlação entre Anomalias e Métricas

Investigar se há correlação entre diferentes métricas nas anomalias detectadas.

In [ ]:
# Mesclar resultados de anomalias de diferentes métricas
anomaly_results = {}

for metric_name in selected_metrics:
    if metric_name in selected_data:
        df = selected_data[metric_name].copy()
        
        # Aplicar detecção de anomalias
        df_anomalies, _ = detect_anomalies_ensemble(
            df, 
            metric_column='value',
            time_column='elapsed_minutes',
            contamination=0.05,
            group_by=['tenant']
        )
        
        # Armazenar resultados
        anomaly_results[metric_name] = df_anomalies

# Criar um DataFrame consolidado com anomalias para diferentes métricas
consolidated_anomalies = None

for metric_name, df in anomaly_results.items():
    # Selecionar apenas colunas relevantes
    temp_df = df[['tenant', 'phase', 'datetime', 'elapsed_minutes', 'value', 'is_anomaly']]
    temp_df = temp_df.rename(columns={
        'value': f'{metric_name}_value', 
        'is_anomaly': f'{metric_name}_anomaly'
    })
    
    # Mesclar com o DataFrame consolidado
    if consolidated_anomalies is None:
        consolidated_anomalies = temp_df
    else:
        consolidated_anomalies = pd.merge(
            consolidated_anomalies,
            temp_df,
            on=['tenant', 'phase', 'datetime', 'elapsed_minutes'],
            how='outer'
        )

# Verificar o DataFrame resultante
if consolidated_anomalies is not None:
    print(f"Formato do DataFrame consolidado: {consolidated_anomalies.shape}")
    display(consolidated_anomalies.head())
else:
    print("Nenhum resultado de anomalia disponível para análise.")

In [ ]:
# Analisar a correlação entre anomalias em diferentes métricas
if consolidated_anomalies is not None and len(selected_metrics) >= 2:
    # Criar tabela de contingência para anomalias
    anomaly_columns = [f'{metric}_anomaly' for metric in selected_metrics]
    contingency_table = pd.crosstab(
        consolidated_anomalies[anomaly_columns[0]], 
        consolidated_anomalies[anomaly_columns[1]],
        rownames=['Anomalia em ' + selected_metrics[0]],
        colnames=['Anomalia em ' + selected_metrics[1]]
    )
    
    # Exibir tabela de contingência
    print(f"Tabela de Contingência para Anomalias em {selected_metrics[0]} vs {selected_metrics[1]}:")
    display(contingency_table)
    
    # Calcular coeficiente phi (correlação para variáveis binárias)
    from scipy.stats import contingency
    
    try:
        chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
        n = contingency_table.sum().sum()
        phi = np.sqrt(chi2 / n)
        
        print(f"\nCoeficiente Phi (correlação entre anomalias): {phi:.4f}")
        print(f"Valor-p: {p:.4f} ({'Significativo' if p < 0.05 else 'Não significativo'} ao nível de 5%)")
    except Exception as e:
        print(f"Erro ao calcular estatísticas: {str(e)}")

## 10. Conclusões e Próximos Passos

Este notebook demonstrou diferentes técnicas de detecção de anomalias que podem ser aplicadas a experimentos de noisy neighbors:

1. **Isolation Forest**: Eficiente para detectar pontos isolados que se desviam do comportamento normal
2. **Local Outlier Factor**: Sensível a anomalias locais baseadas em densidade
3. **Detecção de Pontos de Mudança**: Identifica mudanças significativas no comportamento da série temporal
4. **Detecção de Mudanças de Padrão**: Analisa mudanças em padrões multivariados usando clustering
5. **Ensemble de Algoritmos**: Combina diferentes abordagens para uma detecção mais robusta

### Próximos Passos:

- Validar as anomalias detectadas com especialistas de domínio
- Correlacionar as anomalias com eventos específicos no cluster Kubernetes
- Implementar métricas de avaliação para medir a precisão da detecção
- Expandir a análise para incluir mais métricas e relações entre tenants
- Integrar técnicas de explicabilidade para entender melhor as causas das anomalias